<a href="https://colab.research.google.com/github/JRasmusBm/chatbot-
epsilon/blob/master/Trainer.ipynb" target="_parent"><img
src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In
Colab"/></a>

# This is the file in which we perform training of the NN

# Load
Data

## In Colab

In [63]:
#from google.colab import files
#uploaded = files.upload()
#file_name = "amazon_cells_labelled.txt"
#uploaded[file_name].decode("utf-8")

## Locally

In [64]:
data_folder = "../../data"
file_name = f"{data_folder}/amazon_cells_labelled.txt"
json_file = f"{data_folder}/amazon_cells_labelled.json"

# Import code (from TA)

# Imports

In [83]:
from torchtext import data
import torch
import json
import time

# Extract Data

First, we create lists of labels and sentences. The indices in
the one
correspond to those in the other. Due to restrictions in torchtext,
write it as
json to disk.

In [66]:
with open(file_name) as f:
    contents = f.read()
labels = []
sentences = []
for line in (l for l in contents.split("\n") if l):
    labels.append(int(line[-1]))
    sentences.append(str.strip(line[:-1]).replace(
        ".", "").replace(
        "!", "").replace(
        "?", "").replace(
        ",", "").split(" "))
data_json = [
    dict(label=label, text=text)
    for label, text in zip(labels, sentences)
]
with open(json_file, "w") as f:
    text = "\n".join(json.dumps(line) for line in data_json)
    f.write(text)
    

## Validate

In [67]:
with open(json_file) as f:
    json_written = [json.loads(line) for line in f.read().split("\n")]
    for line in json_written:
        if line["label"] not in [0, 1]:
            print(line)

Now we convert the lists of labels into a torchtext dataset.

In [68]:
import random
from torch.utils.data.dataset import random_split
SEED = 1234


TEXT = data.Field(sequential=True)
LABEL = data.LabelField(dtype = torch.float)
fields=dict(
    text=("text", TEXT), 
    label=("label", LABEL),
)
from IPython.core.debugger import set_trace
dataset = data.TabularDataset(
    path=json_file,
    format="json",
    fields=fields,
) 
#help(dataset)
training_data, test_data, validation_data = dataset.split(
    split_ratio=[0.7, 0.2, 0.1],
    random_state=random.seed(SEED)
)

## Validate

In [69]:
print(f"Length (Training Data): {len(training_data)}")
print(f"Length (Test Data): {len(test_data)}")
print(f"Length (Validation Data): {len(validation_data)}")

Length (Training Data): 700
Length (Test Data): 100
Length (Validation Data): 200


# Build Vocab

In [70]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(training_data, max_size = MAX_VOCAB_SIZE)
LABEL.build_vocab(training_data)

## Validate

In [71]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")
print(TEXT.vocab.freqs.most_common(10))

Unique tokens in TEXT vocabulary: 1828
Unique tokens in LABEL vocabulary: 2
[('the', 300), ('and', 201), ('I', 195), ('is', 175), ('a', 143), ('it', 141), ('to', 136), ('phone', 105), ('this', 100), ('my', 86)]


# Create Iterators

In [75]:
BATCH_SIZE = 64

# Use GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

training_iterator, validation_iterator, test_iterator = data.BucketIterator.splits(
    (training_data, validation_data, test_data), 
    batch_size = BATCH_SIZE,
    sort_key = lambda x: len(x.text),
    sort_within_batch = False,
    device = device)

# Build RNN

In [77]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text):
        embedded = self.embedding(text)
        output, hidden = self.rnn(embedded)
        assert torch.equal(output[-1,:,:], hidden.squeeze(0))
        return self.fc(hidden.squeeze(0))

# Instantiate Model

In [78]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1

model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)

## Validate

In [79]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 274,705 trainable parameters


# Train Model

In [81]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [80]:
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
criterion = nn.BCEWithLogitsLoss()
model = model.to(device)
criterion = criterion.to(device)

def train(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.train()
    for batch in iterator:
        optimizer.zero_grad()
        predictions = model(batch.text).squeeze(1)
        loss = criterion(predictions, batch.label)
        acc = binary_accuracy(predictions, batch.label)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [82]:
def evaluate(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.eval()
    with torch.no_grad():
        for batch in iterator:
            predictions = model(batch.text).squeeze(1)
            loss = criterion(predictions, batch.label)
            acc = binary_accuracy(predictions, batch.label)
            epoch_loss += loss.item()
            epoch_acc += acc.item()
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [84]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

# Run Training

In [ ]:
N_EPOCHS = 5

best_validation_loss = float('inf')

for epoch in range(N_EPOCHS):
    start_time = time.time()
    training_loss, training_acc = train(model, training_iterator, optimizer, criterion)
    validation_loss, validation_acc = evaluate(model, validation_iterator, criterion)
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    if validation_loss < best_validation_loss:
        best_validation_loss = validation_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\ttraining Loss: {training_loss:.3f} | training Acc: {training_acc*100:.2f}%')
    print(f'\t Val. Loss: {validation_loss:.3f} |  Val. Acc: {validation_acc*100:.2f}%')